In [1]:
%matplotlib inline

# import resource 
# resource.setrlimit(resource.RLIMIT_NOFILE, (1000, -1))
# import basic functionality / libs
import sys
import os
import numpy as np
import pandas as pd
import json
import scipy
import pickle

# Import magic commands for jupyter notebook 
# - autoreloading a module
# - profiling functions for memory usage and scripts
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
%load_ext memory_profiler

sys.path.append('../../../')
# import fragility related modules
import dnn
from dnn.base.utils.log_error import initialize_logger
from dnn.base.constants.config import Config
# from dnn.io.loadsimdataset import LoadSimDataset
# from dnn.io.loadresults import LoadResults

# import basic plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [3]:
# set configurations
patient = 'id001_bt'
patients = ['id001_ac', 'id002_cj', 'id008_gc', 'id010_js', 
            'id011_ml', 'id012_pc', 'id013_pg']
patient = 'id013_pg'
output_folder = os.path.join(os.path.expanduser("~"), 'Documents', 'figures', 
        'exp001-008_dnn', patient)

# which experimental set up do we want to compare with?
expname = 'exp001'

#  where to find the raw data for this patient
rawdatadir = os.path.join('/Volumes/ADAM LI/rawdata/old/',  patient)

# configuration and plotting
config = Config(output_base=output_folder) 
config.figures.MATPLOTLIB_BACKEND="qt5" #"Qt4Agg" #"inline"
config.figures.SHOW_FLAG=True

# real data results directory
resultsdir = os.path.join('/Volumes/ADAM LI/pydata/', 'dnn/output/', expname, patient)


In [ ]:
output_folder = os.path.join(os.path.expanduser("~"), 'Documents', 'figures', 
        'exp001-005_dnn')
expname = 'exp003'
print(expname)
loss = np.zeros((len(patients), 300))
acc = np.zeros(loss.shape)
val_acc = np.zeros(loss.shape)
val_loss = np.zeros(loss.shape)

# go through all patients
for idx, pat in enumerate(patients):
    # real data results directory
    resultsdir = os.path.join('/Volumes/ADAM LI/pydata/', 'dnn/output/', expname, pat)
    resultfiles = [f for f in os.listdir(resultsdir) if f.endswith('.pkl')]
    
    # extract data from each training session
    for _file in resultfiles:
        histfile = os.path.join(resultsdir, _file)

        # load in the history file
        with open(histfile, "rb") as f:
            history = pickle.load(f)
        print(history.keys())
        print(len(history['loss']))
        
        loss[idx, :] = history['loss']
        acc[idx, :] = history['acc']
        val_acc[idx,:] = history['val_acc']
        val_loss[idx,:] = history['val_loss']

In [ ]:
# plot the training loss and accuracy
N = np.arange(0, loss.shape[1])
sns.set(font_scale=2)
plt.style.use("ggplot")
fontdict = {'fontsize': 32}

# define mean and error region
all_loss = []
all_acc = []
all_val_loss = []
all_val_acc = []

plt.figure(figsize=(20,20))
axes = plt.gca()
for idx, pat in enumerate(patients):
    all_loss.append(loss[idx,:])
    all_acc.append(acc[idx,:])
    all_val_loss.append(val_loss[idx,:])
    all_val_acc.append(val_acc[idx,:])

y = np.mean(np.array(all_loss), axis=0)
error = np.std(all_loss, axis=0)
plt.plot(N, y, label="train_loss")
plt.fill_between(N, y-error, y+error, alpha=0.5)

y = np.mean(np.array(all_acc), axis=0)
error = np.std(all_acc, axis=0)
plt.plot(N, y, label="train_acc")
plt.fill_between(N, y-error, y+error, alpha=0.5)

y = np.mean(all_val_loss, axis=0)
error = np.std(all_val_loss, axis=0)
plt.plot(N, y, label="val_loss")
plt.fill_between(N, y-error, y+error, alpha=0.5)

y = np.mean(all_val_acc, axis=0)
error = np.std(all_val_acc, axis=0)
plt.plot(N, y, label="val_acc")
plt.fill_between(N, y-error, y+error,
    alpha=0.5, edgecolor='#CC4F1B', facecolor='#FF9848')
plt.title("Training Iteration {}: 12-VGG-CNN".format(expname), fontsize=30)
plt.xlabel("Epoch #", fontsize=30)
plt.ylabel("Loss/Accuracy", fontsize=30)
plt.legend()
axes.autoscale(tight=True)
# plt.tight_layout()
figname=os.path.join(output_folder, '.pdf'.format(expname))
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
plt.savefig(figname, format='pdf')
plt.show()